## Descriptions of data scraped in the end

Google Search Results: Title, text of article, published date
- "Benin Bronzes Restitution" had lots of search results that were articles with 64 600 results in total we fetched 172 unique search result web page contents. 

- "Tendaguru Dinosaurs Restitution" only had 2370 google results in total, many of which were not news articles. We fetched 313 search result contents. 

- Egyption artefacts: Sometimes these are mentioned all together, so combining them we have 468 unique search result web page contents.
    - "nefertiti bust restitution" has 36 400 search results fetched 1309
    - "rosetta stone restitution" has 763 000 search results fetched 1345
    - "Dendera zodiac ceiling restitution" has 4700 search results 1314


Google Scholar: Title, authors, published date, abstract, citing articles + other scholarly objects
- benin bronzes restitution there were 1080 hits on Google scholar, we fetched 995 of those
- tendaguru dinosaurs there are only 8 results! We fetched all 8
- Egyption artefacts
    - Nefertiti Bust restitution 237 we fetched all 237
    - rosetta stone restitution 1 080 I could only get 1000 then I was blocked
    - Dendera zodiac ceiling restitution has 40 we fetched all 40

In [ ]:
Twitter: 

In [1]:
import json

In [80]:
import base64
import trafilatura

In [81]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
from urllib.parse import urlparse

from requests_html import HTML
from requests_html import HTMLSession
import pandas as pd

In [82]:
from htmldate import find_date

In [83]:
from bs4 import BeautifulSoup
import requests, urllib.parse
import lxml

def get_extracted_data_from_url(url):

    headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }
    response = requests.get(url, headers=headers).text

    soup = BeautifulSoup(response, 'lxml')
    articles = []
    for container in soup.findAll('div', class_='tF2Cxc'):
        article = {}
        article['title'] = container.find('h3', class_='LC20lb DKV0Md').text
        try:
            article['datesum'] = container.find('span', class_='f').text
        except AttributeError:
            pass
        article['link'] = container.a['href']
        articles.append(article)

    return articles, soup.select_one('a#pnnext')


def scrape(query, next_page_node):
    
    if next_page_node is None:
        articles, next_page_node = get_extracted_data_from_url(
            'https://google.com/search?hl=en-US&q={}'.format(query))
        data = articles
    else:
        data = []
    
    for i in range(15):
        # scraping breaks beyond 15 pages at a time
        try:
            next_page_url = urllib.parse.urljoin('https://www.google.com', next_page_node['href'])
            articles, next_page_node = get_extracted_data_from_url(next_page_url)
            data = data+articles
        except TypeError:
            print('scraping stopped at {}th page'.format(i))
            break
    return data, next_page_node

In [84]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [85]:
# this seems to work for all, which is nice but need to get metadata as well

def beautifulsoup_extract_text_fallback(response_content):
    
    '''
    This is a fallback function, so that we can always return a value for text content.
    Even for when both Trafilatura and BeautifulSoup are unable to extract the text from a 
    single URL.
    '''
    
    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()
    

def extract_text_from_single_web_page(url):
    
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments = False,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
        #return json_output
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan

In [86]:
def get_text_frm_url(x):
    #ans = get_source(x)
    #page_html = ans.content.decode("utf-8")
    #soup = BeautifulSoup(page_html, 'html.parser')
    try:
        text = extract_text_from_single_web_page(url=x)
    except:
        text = None
        print(x)
    return text

In [87]:
def get_html_date(x):
    try:
        date = find_date(x)
        return date
    except:
        return None

In [88]:
#if its an important media cite it is likely to return these
def get_more_meta(df):
    
    soup = BeautifulSoup(df['link'])
    try:
        data = json.loads(soup.find('script', type='application/ld+json').text)
        datePublished = data['datePublished']
        publisher = data['publisher']
        author = data['author']
        
    except:
        datePublished = 'unknown'
        publisher = 'unknown'
        author = 'unknown'
    df['datePublished'] = datePublished
    df['publisher'] = publisher
    df['author'] = author
    
    return df

# Get Google Search URLs

We dont want to scrape too many at the same time, so what we will do is 10 pages at a time and process them inbetween (fetch the article content) before we scrape google again for the next 10 pages

**"Benin Bronzes Restitution"** had lots of search results that were articles with 64 600 results in total

**"Tendaguru Dinosaurs Restitution"** only had 2370 google results in total, many of which were not news articles. We may have to do a bit of extra cleaning here to make sure they are media articles

Sometimes these are mentioned all together, so I will have to check for duplicates accross all of them:

**"nefertiti bust restitution"** has 36 400 search results fetched 1309

**"rosetta stone restitution"** has 763 000 search results fetched 1345

**"Dendera zodiac ceiling restitution"** has 4700 search results 1314

Loops and stuff were breaking/getting blocked to it is very manual

In [179]:
search_term = 'Dendera zodiac ceiling restitution'

In [180]:
scraped1, next_page_node = scrape(search_term, None)

scraping stopped at 5th page


In [181]:
scraped2, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [182]:
scraped3, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [183]:
scraped4, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [184]:
scraped5, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [185]:
scraped6, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [186]:
scraped7, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [187]:
scraped8, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [188]:
scraped9, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [189]:
scraped10, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [190]:
scraped11, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [191]:
scraped12, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [192]:
scraped13, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [193]:
scraped14, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [194]:
scraped15, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 4th page


In [211]:
scraped16, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 5th page


In [212]:
scraped17, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 5th page


In [213]:
scraped18, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 5th page


In [214]:
scraped19, next_page_node = scrape(search_term, next_page_node)

scraping stopped at 6th page


In [197]:
scraped = scraped1 + scraped2 + scraped3 + scraped4 + scraped5 + scraped6 + scraped7 + scraped8 +scraped9 +scraped10 +scraped11 + scraped12 + scraped13 + scraped14 + scraped15

In [215]:
scraped = scraped + scraped16 + scraped17 + scraped18 + scraped19

In [216]:
len(scraped)

1314

In [178]:
rosetta = scraped

In [146]:
nefer = scraped

In [217]:
dendera = scraped

In [218]:
df = pd.DataFrame(rosetta + nefer + dendera)

## Check for duplicate article links in the scrape and remove them

In [220]:
df.shape

(3714, 2)

In [221]:
deduped = df.drop_duplicates()

In [222]:
deduped.shape

(468, 2)

We scraped lots of pages but a lot of the search results were the same article. If it were not the same article, the google search title and link to article would not be the same

- Benin Bronze 1700 -> 172
- Tendaguru Dinosours 1309 -> 122
- Egypt stuff 3714 -> 468

In [223]:
#write to file so we dont lose them
deduped.to_csv("eg_google_titlelink468.csv")

# Get Article content and dates and extra info if possible

So when I tried to run the 140 result df, it broke and I gor nothing. So I am being extra careful running everything piece by piece so that it will work. First I divided the df into 9 bits (groups of 20). The first run did not take long at all for the zero df, will see how the others go

In [36]:
import numpy as np

In [224]:
df = deduped

In [225]:
n = 20
dfs = []
for g, df in deduped.groupby(np.arange(len(deduped)) // n):
    dfs.append(df)

In [226]:
len(dfs)

24

In [247]:
zero = dfs[18]
zero['text'] = zero['link'].apply(lambda x: get_text_frm_url(x))
zero['html_date'] = zero['link'].apply(lambda x: get_html_date(x))

In [248]:
one = dfs[19]
one['text'] = one['link'].apply(lambda x: get_text_frm_url(x))
one['html_date'] = one['link'].apply(lambda x: get_html_date(x))

In [249]:
two = dfs[20]
two['text'] = two['link'].apply(lambda x: get_text_frm_url(x))
two['html_date'] = two['link'].apply(lambda x: get_html_date(x))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [250]:
three = dfs[21]
three['text'] = three['link'].apply(lambda x: get_text_frm_url(x))
three['html_date'] = three['link'].apply(lambda x: get_html_date(x))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [251]:
four = dfs[22]
four['text'] = four['link'].apply(lambda x: get_text_frm_url(x))
four['html_date'] = four['link'].apply(lambda x: get_html_date(x))

http://www.gshock.cmg.co.th/download/NDI5bXNUTFE4cUtNWDBDMzNrRk9jKzJLSnVyNjBsTzhnNDludkhOSVUrU2VxWFlLWUd6TFRFajBaNVh5eWZUcGVtd1JrNkZXRlFUMEVuOVVDSzIyanZlWXhCcVJEY0VNM1hKSkpwSG5GRXc9_pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [252]:
five = dfs[23]
five['text'] = five['link'].apply(lambda x: get_text_frm_url(x))
five['html_date'] = five['link'].apply(lambda x: get_html_date(x))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [233]:
part1 = pd.concat([zero, one, two, three, four, five])

In [239]:
part2 = pd.concat([zero, one, two, three, four, five])

In [246]:
part3 = pd.concat([zero, one, two, three, four, five])

In [253]:
part4 = pd.concat([zero, one, two, three, four, five])

In [254]:
final_df = pd.concat([part1, part2, part3,part4])

In [255]:
final_df.shape

(468, 4)

In [256]:
final_df.to_csv('eg_google_content468.csv')

In [257]:
final_df

,title,link,text,html_date
0,Should the British Return the Stolen Rosetta S...,https://www.papermag.com/going-going-gone-2632...,If 2018 was a year plagued by inconvenient tru...,2019-03-18
1,The British Museum's claims to the Rosetta Sto...,http://www.elginism.com/similar-cases/the-brit...,"October 12, 2008\nThe British Museum’s claims ...",2008-10-12
2,Egyptian museum calls for Rosetta Stone to be ...,https://www.telegraph.co.uk/news/2018/11/06/eg...,Egyptian museum calls for Rosetta Stone to be ...,2018-11-06
3,The battle for the Rosetta Stone | The Indepen...,https://www.independent.co.uk/news/uk/this-bri...,The battle for the Rosetta Stone\nThings are l...,2013-10-26
4,Repatriation; restitution | The Rosetta Stone:...,https://jdownsrosetta.wordpress.com/category/r...,Egypt’s Antiquities Caught in the Revolution\n...,2011-04-06
...,...,...,...,...
3130,2021 BOUTIQUE RIVER CRUISES - Reisebüro Passag...,https://passage-kontor.de/fileadmin/kataloge/U...,]/Index[18948 30]/Info 18947 0 R/Length 100/Pr...,None
3134,FINAL STUDY - ENPI CBC Med,http://www.enpicbcmed.eu/sites/default/files/c...,U!{2&�܋xbNA � \nz�a�L�=�����?J��ۑ>��9�...,None
3135,Louvre - Study in China 2021 - Wikipedia Engli...,https://www.duhocchina.com/wiki/en/Louvre,- Arabic\n- English\n- Deutsch\n- French\n- It...,2006-05-14
3568,Louvre Gives In To Demand Of Zahi Hawas For Th...,https://www.africavenir.org/fr/newsdetails/arc...,Louvre Gives In To Demand Of Zahi Hawas For Th...,2009-01-01


# Clean Article text and load to dataframe

In [258]:
import nltk
import pandas as pd
import CleanTweets as ct

In [259]:
df = pd.read_csv('eg_google_content468.csv', index_col=0)

In [260]:
df.head()

,title,link,text,html_date
0,Should the British Return the Stolen Rosetta S...,https://www.papermag.com/going-going-gone-2632...,If 2018 was a year plagued by inconvenient tru...,2019-03-18
1,The British Museum's claims to the Rosetta Sto...,http://www.elginism.com/similar-cases/the-brit...,"October 12, 2008\nThe British Museum’s claims ...",2008-10-12
2,Egyptian museum calls for Rosetta Stone to be ...,https://www.telegraph.co.uk/news/2018/11/06/eg...,Egyptian museum calls for Rosetta Stone to be ...,2018-11-06
3,The battle for the Rosetta Stone | The Indepen...,https://www.independent.co.uk/news/uk/this-bri...,The battle for the Rosetta Stone\nThings are l...,2013-10-26
4,Repatriation; restitution | The Rosetta Stone:...,https://jdownsrosetta.wordpress.com/category/r...,Egypt’s Antiquities Caught in the Revolution\n...,2011-04-06


In [261]:
df = ct.clean_tweets(df,'text','lemma_text',
                general_clean=True,lemma=True,stem=False,remove_tag=False,remove_mention=False,
                remove_emoji=False, remove_stopword=True,min_length=1, untokenized_return=True)

time taken to clean tweets: 92.81211709976196s. Use the [lemma_text] column to perform your analysis/modeling on


In [262]:
df = ct.clean_tweets(df,'text','stem_text',
                general_clean=True,lemma=False,stem=True,remove_tag=False,remove_mention=False,
                remove_emoji=False, remove_stopword=True,min_length=1, untokenized_return=True)

time taken to clean tweets: 15.491995096206665s. Use the [stem_text] column to perform your analysis/modeling on


In [263]:
df.to_csv('eg_google_content468_cleaned.csv')

# Playground

In [87]:
def ordinary(soup):

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.body.get_text(separator=' ')

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

In [131]:
def bcc_get_p(soup):
    # for BBC articles
    # retrieve all of the paragraph tags
    #paragraphs = soup.find('article').find_all('p') 
    paragraphs = soup.find_all('p') 
    text = ' '.join(paragraph.text for paragraph in paragraphs)
    headings = soup.find_all('h1')
    print(headings)
    text2 = ' '.join(heading.text for heading in headings)
    full_text = text + text2
    return full_text

In [120]:
# BCC and CNN
article = {}
article['publisher'] = data['publisher']
article['author'] = data['author']
article['url'] = data['url']
article['datePublished'] = data['datePublished']
article['text'] = ordinary(soup)

In [142]:
bcc_get_p(soup)

[<h1 class="page-header__title"><span>Effort to Return Benin Bronzes to Africa Remains Ongoing Challenge</span>
</h1>]


" LONDON - The promise to return several Benin Bronzes from three Western institutions to the former Kingdom of Benin in Nigeria was celebrated by many. But returning all the artifacts looted by British soldiers 125 years ago will continue to be a challenge. In recent weeks, a university in Scotland and museums in Germany and Britain pledged to repatriate the Benin Bronzes they own.\xa0\xa0\xa0 The restitution is hugely symbolic to Timothy Awoyemi, a British-born Nigerian who helped repatriate two Benin Bronzes in 2014 from a private collector whose grandfather had been part of the 19th century looting. Awoyemi says he was elated when he heard about the latest returns.\xa0 “They stole it so it makes me happy, because the stolen artifacts are going to be returned back to where they rightly belong,” he said.\xa0\xa0 British soldiers looted the Kingdom of Benin in what is currently Edo state in Nigeria, during a punitive military expedition in 1897. The high valued plaques, masks and scul

In [143]:
ordinary(soup)

"Skip to main content\nOpen main navigation\nLive TV\nFull Schedule\nAll Programs\nLive Radio\nFull Schedule\nAll Programs\nUnited States\nUS Politics\nImmigration\nAll About America\nScience & Health\nSilicon Valley & Technology\nWorld\nAfrica\nThe Americas\nEast Asia Pacific\nEurope\nExtremism Watch\nMiddle East\nSouth & Central Asia\nVOA News on China\nVOA News on Iran\nSections\nArts & Culture\nDay in Photos\nEconomy & Business\nPress Freedom\nPlugged in with Greta Van Susteren\nVOA StudentU\nVOA Connect\nRefugees\nFEATURED\nUS News\nPress Freedom\nVOA News on Iran\nCOVID-19 Pandemic\nSearch\nSearch VOA News\nSearch\nLang\nLive TV\nLive TV\nFull Schedule\nAll Programs\nLive Radio\nLive Radio\nFull Schedule\nAll Programs\nEnglish\nEnglish\nvoanews.com\nLearning English\nlearningenglish.voanews.com\nEastern & Central Europe\nShqip\nzeriamerikes.com\nBosanski\nba.voanews.com\nΕλληνικά\ngr.voanews.com\nМакедонски\nmk.voanews.com\nSrpski\nglasamerike.net\nУкраїнська\nukrainian.voanews.c

In [112]:
article

{'publisher': {'@type': 'Organization',
  'name': 'CNN',
  'logo': {'@type': 'ImageObject',
   'url': 'https://dynaimage.cdn.cnn.com/cnn/q_auto,h_60/https%3A%2F%2Fi2.cdn.turner.com%2Fcnn%2F2017%2Fimages%2F09%2F14%2Flogo-cnnstyle.png'}},
 'author': {'@type': 'Person', 'name': 'Kieron Monks, CNN'},
 'url': 'https://www.cnn.com/style/article/benin-bronzes-germany-restitution/index.html',
 'datePublished': '2021-04-30T03:03:12Z',
 'text': "Skip to main content\nOpen main navigation\nLive TV\nFull Schedule\nAll Programs\nLive Radio\nFull Schedule\nAll Programs\nUnited States\nUS Politics\nImmigration\nAll About America\nScience & Health\nSilicon Valley & Technology\nWorld\nAfrica\nThe Americas\nEast Asia Pacific\nEurope\nExtremism Watch\nMiddle East\nSouth & Central Asia\nVOA News on China\nVOA News on Iran\nSections\nArts & Culture\nDay in Photos\nEconomy & Business\nPress Freedom\nPlugged in with Greta Van Susteren\nVOA StudentU\nVOA Connect\nRefugees\nFEATURED\nUS News\nPress Freedom\nVO

My update:
    